# Test Results

In this notebook the test results for the different YOLO models are generated

### import

In [1]:
from ultralytics import YOLO
import pandas as pd
import time
import os

## generate results

In [2]:
def get_test_inference(model_path):
    model = YOLO(model_path)

    # Run inference
    results = model.predict(
        source="./datasets/YOLODataset/images/test",
        conf=0.3,       
        save=False,
        stream=True,
        batch=1,
        device=[3],
        verbose=False
    )
    # Initialize an empty list to store results
    data = []
    
    # Start time
    start_time = time.time()
    
    # Process results
    for result in results:
        # Initialize an empty list for detections in the current image
        detections = []
    
        boxes = result.boxes  # Bounding box outputs
        if boxes is not None:
            for box in boxes:
                # Extract information for each detected object
                bbox = box.xyxy[0].tolist()  # Bounding box (x1, y1, x2, y2)
                confidence = box.conf[0].item()  # Confidence score
                class_label = int(box.cls[0].item())  # Class label
    
                # Add to the results list
                detections.append({
                    "x1": bbox[0],
                    "y1": bbox[1],
                    "x2": bbox[2],
                    "y2": bbox[3],
                    "confidence": confidence,
                    "class_label": class_label
                })
    
        data.append({
            "path": result.path.split("/")[-1],
            "detections": detections  # Store all detections for the image
        })
    
    # End time
    end_time = time.time()
    
    # Calculate elapsed time
    elapsed_time = end_time - start_time

    # Calculate FPS
    fps = 2877 / elapsed_time
    
    print(f"Time needed to process results: {elapsed_time:.2f} seconds")
    print(f"FPS: {fps:.2f}")
    
    # Convert results to a DataFrame
    return pd.DataFrame(data)

In [3]:
models  = [[os.path.join(f.path, 'weights/best.pt'), f.path.split('/')[-1].replace("train_","")] 
         for f in os.scandir('./ball-tracking/') 
         if f.is_dir() and "train" in str(f.path)]
models 

[['./ball-tracking/train_yolov10n/weights/best.pt', 'yolov10n'],
 ['./ball-tracking/train_yolov9s/weights/best.pt', 'yolov9s'],
 ['./ball-tracking/train_tuned_yolov5su/weights/best.pt', 'tuned_yolov5su'],
 ['./ball-tracking/train_yolov10s/weights/best.pt', 'yolov10s'],
 ['./ball-tracking/train_yolov10m/weights/best.pt', 'yolov10m'],
 ['./ball-tracking/train_tuned_yolo11s/weights/best.pt', 'tuned_yolo11s'],
 ['./ball-tracking/train_yolov8s/weights/best.pt', 'yolov8s'],
 ['./ball-tracking/train_yolov5su/weights/best.pt', 'yolov5su'],
 ['./ball-tracking/train_yolov10l/weights/best.pt', 'yolov10l'],
 ['./ball-tracking/train_yolo11s/weights/best.pt', 'yolo11s'],
 ['./ball-tracking/train_yolov10x/weights/best.pt', 'yolov10x']]

In [4]:
results = []
for path,model in models:
    print(model)
    df = get_test_inference(path)
    df['model'] = model
    results.append(df)

yolov10n
Time needed to process results: 33.04 seconds
FPS: 87.08
yolov9s
Time needed to process results: 38.69 seconds
FPS: 74.35
tuned_yolov5su
Time needed to process results: 24.54 seconds
FPS: 117.22
yolov10s
Time needed to process results: 28.37 seconds
FPS: 101.41
yolov10m
Time needed to process results: 31.34 seconds
FPS: 91.81
tuned_yolo11s
Time needed to process results: 27.57 seconds
FPS: 104.35
yolov8s
Time needed to process results: 23.28 seconds
FPS: 123.56
yolov5su
Time needed to process results: 24.42 seconds
FPS: 117.80
yolov10l
Time needed to process results: 36.56 seconds
FPS: 78.68
yolo11s
Time needed to process results: 27.59 seconds
FPS: 104.27
yolov10x
Time needed to process results: 38.70 seconds
FPS: 74.34


In [5]:
df = pd.concat(results)

In [6]:
df.sample(10)

,path,detections,model
2052,TrackNet_game4_Clip7_241.jpg,"[{'x1': 505.08941650390625, 'y1': 260.53109741...",yolov10l
761,New_Video_3_clip_8_10.jpg,"[{'x1': 455.6346435546875, 'y1': 303.473388671...",yolo11s
1196,New_Video_4_clip_7_255.jpg,"[{'x1': 368.630126953125, 'y1': 293.2926940917...",tuned_yolo11s
2675,TrackNet_game8_Clip9_130.jpg,"[{'x1': 830.2001953125, 'y1': 503.255737304687...",yolov10s
968,New_Video_3_clip_8_44.jpg,"[{'x1': 618.78125, 'y1': 182.66943359375, 'x2'...",yolov10s
729,New_Video_3_clip_7_300.jpg,"[{'x1': 665.705078125, 'y1': 288.806640625, 'x...",yolov8s
397,New_Video_2_clip_7_169.jpg,[],tuned_yolov5su
2374,TrackNet_game6_Clip4_160.jpg,"[{'x1': 601.8062744140625, 'y1': 189.192535400...",yolov10m
2403,TrackNet_game6_Clip4_187.jpg,"[{'x1': 912.176025390625, 'y1': 376.1239929199...",yolov10m
1427,New_Video_5_clip_6_183.jpg,"[{'x1': 626.8560791015625, 'y1': 269.064117431...",yolov8s


In [ ]:
df.to_csv('../results/Pre_Results_YOLO.csv', index=False)